# **Robótica Industrial - Aula 07 - Cinemática Inversa e de Velocidade em 3D - Transformações Homogêneas Simbólicas** 
Prof. Lucas Vago Santana

*Obs: O objetivo do código é ser didático. Portanto, ele pode não possuir a implementação mais eficiente.*

In [6]:
import numpy as np
import sympy as sym

######################################################################################
############################### PRINT SIMBÓLICO EM LATEX #############################
######################################################################################
# Fonte: https://gist.github.com/snowdj/e7de73023d9ed8b79dcce9a46a57dae8

def custom_latex_printer(exp, **options):
    from google.colab.output._publish import javascript
    url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
    javascript(url=url)
    return sym.printing.latex(exp, **options)

sym.init_printing(use_latex="mathjax", latex_printer=custom_latex_printer)

######################################################################################
####################### FUNÇÕES DE TRANSFORMAÇÃO HOMOGÊNEA ###########################
######################################################################################
#Cria tranformação homogênea a partir dos parâmetros de Denavit-Hartenberg
def t_DH(theta, d, a, alpha): 
  ca = sym.cos(alpha) # Correção da "gambiarra" sym.cos é o mesmo que π/2 simbólico
  sa = sym.sin(alpha)
  
  ct = sym.cos(theta)
  st = sym.sin(theta)

  Rz =  np.array((  (ct, -st, 0, 0), 
                    (st,  ct, 0, 0),
                    ( 0,   0, 1, 0),
                    ( 0,   0, 0, 1) ))
  
  Tz = np.array((   (1, 0, 0, 0), 
                    (0, 1, 0, 0),
                    (0, 0, 1, d),
                    (0, 0, 0, 1) ))

  Rx =  np.array((  (1,  0,   0, 0), 
                    (0, ca, -sa, 0),
                    (0, sa,  ca, 0),
                    (0,  0,   0, 1) ))
  
  Tx = np.array((   (1, 0, 0, a), 
                    (0, 1, 0, 0),
                    (0, 0, 1, 0),
                    (0, 0, 0, 1) ))
  
  return np.linalg.multi_dot([Rz, Tz, Tx, Rx])

######################################################################################
################################# PYTHON SIMBÓLICO ###################################
######################################################################################
q1, a1, q2, a2, q3, a3 = sym.symbols('q1 a1 q2 a2 q3 a3')

"""
    Criação das coordenadas dos elos do robô, baseando-se nos parâmetros DH
    Parâmetros DH:
          theta     d       a       alpha
    1     q1        a1      0       -90
    2     -90+q2    0       a2      0
    3     90+q3     0       a3      0
"""

T_1_0 = t_DH(q1, a1, 0, -sym.pi/2) # Correção da "gambiarra" sym.pi/2 é o mesmo que π/2 simbólico
T_2_1 = t_DH(-sym.pi/2+q2, 0, a2, 0) 
T_3_2 = t_DH(sym.pi/2+q3, 0, a3, 0)  
T_3_1 = np.linalg.multi_dot([ T_2_1, T_3_2])
T_3_0 = np.linalg.multi_dot([T_1_0, T_2_1, T_3_2])
T_2_0 = np.linalg.multi_dot([T_1_0, T_2_1])

In [8]:
print('T de {1} para {0}:')
Tprint = T_1_0
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

T de {1} para {0}:


ModuleNotFoundError: No module named 'google'

⎡cos(q₁)  0   -sin(q₁)  0 ⎤
⎢                         ⎥
⎢sin(q₁)  0   cos(q₁)   0 ⎥
⎢                         ⎥
⎢   0     -1     0      a₁⎥
⎢                         ⎥
⎣   0     0      0      1 ⎦

In [ ]:
#print('T de {2} para {1}:')
Tprint = T_2_1
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡sin(q₂)   cos(q₂)  0  a₂⋅sin(q₂) ⎤
⎢                                 ⎥
⎢-cos(q₂)  sin(q₂)  0  -a₂⋅cos(q₂)⎥
⎢                                 ⎥
⎢   0         0     1       0     ⎥
⎢                                 ⎥
⎣   0         0     0       1     ⎦

In [ ]:
#print('T de {3} para {2}:')
Tprint = T_3_2
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡-sin(q₃)  -cos(q₃)  0  -a₃⋅sin(q₃)⎤
⎢                                  ⎥
⎢cos(q₃)   -sin(q₃)  0  a₃⋅cos(q₃) ⎥
⎢                                  ⎥
⎢   0         0      1       0     ⎥
⎢                                  ⎥
⎣   0         0      0       1     ⎦

In [ ]:
#print('T de {3} para {1}:')
Tprint = T_3_1
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡cos(q₂ + q₃)  -sin(q₂ + q₃)  0  a₂⋅sin(q₂) + a₃⋅cos(q₂ + q₃) ⎤
⎢                                                             ⎥
⎢sin(q₂ + q₃)  cos(q₂ + q₃)   0  -a₂⋅cos(q₂) + a₃⋅sin(q₂ + q₃)⎥
⎢                                                             ⎥
⎢     0              0        1                0              ⎥
⎢                                                             ⎥
⎣     0              0        0                1              ⎦

In [ ]:
#print('T de {3} para {0}:')
Tprint = T_3_0
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡cos(q₁)⋅cos(q₂ + q₃)  -sin(q₂ + q₃)⋅cos(q₁)  -sin(q₁)  (a₂⋅sin(q₂) + a₃⋅cos(q
⎢                                                                             
⎢sin(q₁)⋅cos(q₂ + q₃)  -sin(q₁)⋅sin(q₂ + q₃)  cos(q₁)   (a₂⋅sin(q₂) + a₃⋅cos(q
⎢                                                                             
⎢   -sin(q₂ + q₃)          -cos(q₂ + q₃)         0        a₁ + a₂⋅cos(q₂) - a₃
⎢                                                                             
⎣         0                      0               0                        1   

₂ + q₃))⋅cos(q₁)⎤
                ⎥
₂ + q₃))⋅sin(q₁)⎥
                ⎥
⋅sin(q₂ + q₃)   ⎥
                ⎥
                ⎦

In [ ]:
#print('T de {2} para {0}:')
Tprint = T_2_0
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡sin(q₂)⋅cos(q₁)  cos(q₁)⋅cos(q₂)  -sin(q₁)  a₂⋅sin(q₂)⋅cos(q₁)⎤
⎢                                                              ⎥
⎢sin(q₁)⋅sin(q₂)  sin(q₁)⋅cos(q₂)  cos(q₁)   a₂⋅sin(q₁)⋅sin(q₂)⎥
⎢                                                              ⎥
⎢    cos(q₂)         -sin(q₂)         0       a₁ + a₂⋅cos(q₂)  ⎥
⎢                                                              ⎥
⎣       0                0            0              1         ⎦

In [ ]:
R0_0=np.array([[1,0,0],
              [0,1,0],
              [0,0,1]])

O0_0=np.array([[0],
               [0],
               [0]])

R0_1=np.array([[sym.cos(q1),0,sym.sin(q1)],
              [sym.sin(q1),0,-sym.cos(q1)],
              [0,1,0]])

O0_1=np.array([[0],
               [0],
               [a1]])


R0_2=np.array([[-sym.sin(q2)*sym.cos(q1),-sym.cos(q1)*sym.cos(q2),sym.sin(q1)],
              [-sym.sin(q1)*sym.sin(q2),-sym.sin(q1)*sym.cos(q2),-sym.cos(q1)],
              [sym.cos(q2),-sym.sin(q2),0]])

O0_2=np.array([[-a2*sym.sin(q2)*sym.cos(q1)],
               [-a2*sym.sin(q1)*sym.sin(q2)],
               [a1+a2*sym.cos(q2)]])

O0_3=np.array([[(-a2*sym.sin(q2)+a3*sym.cos(q2+q3))*sym.cos(q1)],
               [(-a2*sym.sin(q2)+a3*sym.cos(q2+q3))*sym.sin(q1)],
               [a1+a2*sym.cos(q2)+a3*sym.sin(q2+q3)]])
V1=np.array([[0],
             [0],
             [1]])


Tprint = O0_0
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [ ]:
j11=np.cross((R0_0@V1).T,(O0_3-O0_0).T)
Tprint =j11.T
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡(a₂⋅sin(q₂) - a₃⋅cos(q₂ + q₃))⋅sin(q₁) ⎤
⎢                                       ⎥
⎢(-a₂⋅sin(q₂) + a₃⋅cos(q₂ + q₃))⋅cos(q₁)⎥
⎢                                       ⎥
⎣                   0                   ⎦

In [ ]:
j12=np.cross((R0_1@V1).T,(O0_3-O0_1).T)
Tprint =j12.T
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡-(a₂⋅cos(q₂) + a₃⋅sin(q₂ + q₃))⋅cos(q₁)⎤
⎢                                       ⎥
⎢-(a₂⋅cos(q₂) + a₃⋅sin(q₂ + q₃))⋅sin(q₁)⎥
⎢                                       ⎥
⎣     -a₂⋅sin(q₂) + a₃⋅cos(q₂ + q₃)     ⎦

In [ ]:
j13=np.cross((R0_2@V1).T,(O0_3-O0_2).T)
Tprint =j13.T
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡-a₃⋅sin(q₂ + q₃)⋅cos(q₁)⎤
⎢                        ⎥
⎢-a₃⋅sin(q₁)⋅sin(q₂ + q₃)⎥
⎢                        ⎥
⎣    a₃⋅cos(q₂ + q₃)     ⎦

In [ ]:
j21=R0_0@V1
Tprint =j21
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣1⎦

In [ ]:
j22=R0_1@V1
Tprint =j22
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡sin(q₁) ⎤
⎢        ⎥
⎢-cos(q₁)⎥
⎢        ⎥
⎣   0    ⎦

In [ ]:
j23=R0_2@V1
Tprint =j23
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡sin(q₁) ⎤
⎢        ⎥
⎢-cos(q₁)⎥
⎢        ⎥
⎣   0    ⎦

In [ ]:
J=np.array([[j11[0,0],j12[0,0],j13[0,0]],
            [j11[0,1],j12[0,1],j13[0,1]],
            [j11[0,2],j12[0,2],j13[0,1]],
            [j21[0,0],j22[0,0],j23[0,0]],
            [j21[1,0],j22[1,0],j23[1,0]],
            [j21[2,0],j22[2,0],j23[2,0]]])
Tprint =J
Tprint = sym.simplify(Tprint) #Symplifica equação simbólica
Tprint #Exibe no console de saída

⎡(a₂⋅sin(q₂) - a₃⋅cos(q₂ + q₃))⋅sin(q₁)   -(a₂⋅cos(q₂) + a₃⋅sin(q₂ + q₃))⋅cos(
⎢                                                                             
⎢(-a₂⋅sin(q₂) + a₃⋅cos(q₂ + q₃))⋅cos(q₁)  -(a₂⋅cos(q₂) + a₃⋅sin(q₂ + q₃))⋅sin(
⎢                                                                             
⎢                   0                          -a₂⋅sin(q₂) + a₃⋅cos(q₂ + q₃)  
⎢                                                                             
⎢                   0                                     sin(q₁)             
⎢                                                                             
⎢                   0                                    -cos(q₁)             
⎢                                                                             
⎣                   1                                        0                

q₁)  -a₃⋅sin(q₂ + q₃)⋅cos(q₁)⎤
                             ⎥
q₁)  -a₃⋅sin(q₁)⋅sin(q₂ + q₃)⎥
                             ⎥
     -